In [2]:
! pip install kaggle

In [3]:
#configuring the path of kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [4]:
#api to fetch the dataset from kaggle
#just copying and paste from kaggle
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 88% 71.0M/80.9M [00:00<00:00, 93.5MB/s]
100% 80.9M/80.9M [00:00<00:00, 99.1MB/s]


In [5]:
#extract the zip file sentiment140.zip

from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

with ZipFile(dataset, 'r') as zip:
  zip.extractall()
  print('the dataset is extracted')

the dataset is extracted


In [6]:
import numpy as np
import pandas as pd
import re #re stands for 'regular expression' which is used for pattern matching and search through the datas
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer #used for stemming
from sklearn.feature_extraction.text import TfidfVectorizer #used for changing the numerical data in text format
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [7]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [20]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [8]:
# loading the data from the csv file(of kaggle) to our pandas dataframe
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding = 'ISO-8859-1')

In [9]:
# checking the number of rows and columns from twitter_data
twitter_data.shape

(1599999, 6)

In [ ]:
twitter_data.head(5)

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [10]:
# naming the columns and reading the dataset again

column_names = ['target', 'ids', 'date', 'flag', 'user', 'text']
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', names = column_names, encoding = 'ISO-8859-1')
twitter_data.shape

(1600000, 6)

In [ ]:
twitter_data.head(5)

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [11]:
# counting the missing values in the dataset

twitter_data.isnull().sum()

,0
target,0
ids,0
date,0
flag,0
user,0
text,0


In [12]:
# checking the distribution of target column
twitter_data['target'].value_counts()

,count
target,
0,800000
4,800000


In [28]:
twitter_data.replace({'target': {4:'positive'}}, inplace=True)
twitter_data['target'].value_counts()

,count
target,
negative,800000
positive,800000


In [14]:
twitter_data.replace({'target': {0:'negative'}}, inplace=True)
twitter_data['target'].value_counts()

,count
target,
negative,800000
4,800000


stemming:


In [15]:
porter_stemmer = PorterStemmer()

In [23]:
def stemming(content):

  stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
  stemmed_content= stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [porter_stemmer.stem(words) for words in stemmed_content if not words in stopwords.words('english')]
  stemmed_content= ' '.join(stemmed_content)

  return stemmed_content

In [24]:
twitter_data['stemmed_content']= twitter_data['text'].apply(stemming) #takes about 50 fucking minutes to execute this

In [26]:
print(twitter_data['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [29]:
print(twitter_data['target'])

0          negative
1          negative
2          negative
3          negative
4          negative
             ...   
1599995    positive
1599996    positive
1599997    positive
1599998    positive
1599999    positive
Name: target, Length: 1600000, dtype: object


In [30]:
#seperating the data and label

X = twitter_data['stemmed_content'].values
Y = twitter_data['target'].values

In [31]:
print(X)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


**What are Train Data?**

- they are like the data in which the model has studied.
example:- In your examination, it is like the question you have preapred for which might come in the exam.


**What are Test Data?**

- they are like the data in which the model does not know.
Example:- In your examination, its like the questions that you dont know

In [36]:
X_train, X_test, Y_train,  Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)

# X is the tweet and Y is the label
# X_train contains all the training tweets from X that will be stemmed_context
# Y_train contains all the training tweets from Y
# X_test contains test data tweets
# Y_test contains whether it is a positive tweet for all the X_test
# X_train and Y_train are linked with one another so is X_test and Y_test
# test_size=0.2 means how many data points you need for test data, so 0.2 means 20% of my data wil go to my test data and 80% will go to trainig data
# so out of 1.6million tweets 20% will go in X_test data whereas 80% wil go in X_train data
# considering the percentage of the data is in your hands
# statify means i want equal proportion of negative and postive datas

In [37]:
print(X.shape, X_test.shape, X_train.shape)

(1600000,) (320000,) (1280000,)


In [38]:
print(X_train)

['watch saw iv drink lil wine' 'hatermagazin'
 'even though favourit drink think vodka coke wipe mind time think im gonna find new drink'
 ... 'eager monday afternoon'
 'hope everyon mother great day wait hear guy store tomorrow'
 'love wake folger bad voic deeper']


In [39]:
print(X_test)

['mmangen fine much time chat twitter hubbi back summer amp tend domin free time'
 'ah may show w ruth kim amp geoffrey sanhueza'
 'ishatara mayb bay area thang dammit' ...
 'destini nevertheless hooray member wonder safe trip' 'feel well'
 'supersandro thank']


Converting the text data into numerical data


In [40]:
vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train)
# the fit tries to understand what is the nature of the data that are present here
## transform is converting the textual data to numerical data so that the machine can have better understanding

X_test = vectorizer.transform(X_test)

In [41]:
print(X_train)

  (0, 443066)	0.4484755317023172
  (0, 235045)	0.41996827700291095
  (0, 109306)	0.3753708587402299
  (0, 185193)	0.5277679060576009
  (0, 354543)	0.3588091611460021
  (0, 436713)	0.27259876264838384
  (1, 160636)	1.0
  (2, 288470)	0.16786949597862733
  (2, 132311)	0.2028971570399794
  (2, 150715)	0.18803850583207948
  (2, 178061)	0.1619010109445149
  (2, 409143)	0.15169282335109835
  (2, 266729)	0.24123230668976975
  (2, 443430)	0.3348599670252845
  (2, 77929)	0.31284080750346344
  (2, 433560)	0.3296595898028565
  (2, 406399)	0.32105459490875526
  (2, 129411)	0.29074192727957143
  (2, 407301)	0.18709338684973031
  (2, 124484)	0.1892155960801415
  (2, 109306)	0.4591176413728317
  (3, 172421)	0.37464146922154384
  (3, 411528)	0.27089772444087873
  (3, 388626)	0.3940776331458846
  (3, 56476)	0.5200465453608686
  :	:
  (1279996, 390130)	0.22064742191076112
  (1279996, 434014)	0.2718945052332447
  (1279996, 318303)	0.21254698865277746
  (1279996, 237899)	0.2236567560099234
  (1279996, 2910

In [42]:
print(X_test)

  (0, 420984)	0.17915624523539803
  (0, 409143)	0.31430470598079707
  (0, 398906)	0.3491043873264267
  (0, 388348)	0.21985076072061738
  (0, 279082)	0.1782518010910344
  (0, 271016)	0.4535662391658828
  (0, 171378)	0.2805816206356073
  (0, 138164)	0.23688292264071403
  (0, 132364)	0.25525488955578596
  (0, 106069)	0.3655545001090455
  (0, 67828)	0.26800375270827315
  (0, 31168)	0.16247724180521766
  (0, 15110)	0.1719352837797837
  (1, 366203)	0.24595562404108307
  (1, 348135)	0.4739279595416274
  (1, 256777)	0.28751585696559306
  (1, 217562)	0.40288153995289894
  (1, 145393)	0.575262969264869
  (1, 15110)	0.211037449588008
  (1, 6463)	0.30733520460524466
  (2, 400621)	0.4317732461913093
  (2, 256834)	0.2564939661498776
  (2, 183312)	0.5892069252021465
  (2, 89448)	0.36340369428387626
  (2, 34401)	0.37916255084357414
  :	:
  (319994, 123278)	0.4530341382559843
  (319995, 444934)	0.3211092817599261
  (319995, 420984)	0.22631428606830145
  (319995, 416257)	0.23816465111736276
  (319995, 3

training the machine learning model

logistic Regression

In [45]:
model = LogisticRegression (max_iter=1000)
# max_iter = maximum number of iteration the model can go through the data

In [46]:
model.fit(X_train, Y_train)
# X_train is my trainig data tweet
# Y_train is the target for my training data tweets

LogisticRegression(max_iter=1000)

**Model Evaluation**
accuracy score

In [50]:
# accuracy score in the training data

# once the model(upper code) is done learning from the data it will make predictions from the X_train
X_train_prediction = model.predict(X_train)
# predictions(negative or positive) are made and then stored in X_train_prediction model
training_data_accuraccy = accuracy_score(Y_train, X_train_prediction)

In [51]:
print('accuracy data is :', training_data_accuraccy)

accuracy data is : 0.81018984375


that means the accuracy is arounf 81% which means we have trained our model enough to make the accuracy of 81 tweets so remaining 19 tweets can be wrong.

In [52]:
# accuracy score in the test data

X_test_prediction = model.predict(X_test)
test_data_accuraccy = accuracy_score(Y_test, X_test_prediction)

In [53]:
print('accuracy score on test data  is :', test_data_accuraccy)

accuracy score on test data  is : 0.7780375


**Model Acuracy = 77.8%**

If the accuracy score is greater than 75% then it is considered to be a good model

**saving the model, so that you dont have to train the model everytime **

In [59]:
import pickle

filename = 'trained_model.sav' # .sav is a saved file extentsion
pickle.dump(model, open(filename, 'wb'))
# dump stores all the parameters that is learned by the model and will be stored in this file
# 'wb' is writing the the file and b means in the binary format

saving the model= pickle.dump()
loading the model = pickle.load()

*using the saved model for future prediction*

In [61]:
# loading the saved model

# Load the saved model
with open('/content/trained_model.sav', 'rb') as file:
    loaded_model = pickle.load(file)
# 'rb' is reading the the file and b means in the binary format

In [62]:
print(X.shape, X_test.shape, X_train.shape)

(1600000,) (320000, 461488) (1280000, 461488)


In [66]:
# making a new predictions

X_new = X_test[200]
print(Y_test[200])
# as the X_test data has like 320000 data points and from there i am going to print the 200th data point

positive


In [70]:
X_new = X_test[200]
prediction_tweet = model.predict(X_new)

if (prediction_tweet == 'negative'):
  print('The Tweet is negative...')
else:
  print('The Tweet is Positive')

The Tweet is Positive


In [72]:
X_new = X_test[12345]

prediction_tweet = model.predict(X_new)

if (prediction_tweet == 'negative'):
  print('The Tweet is negative...')
else:
  print('The Tweet is Positive')

The Tweet is negative...
